In [1]:
import os
import re
from collections import defaultdict

import pandas as pd

import camelot

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [128]:
dir_item = '/home/jake/Documents/News_Item/'
dir_proj = dir_item + 'fukushima/'
dir_tableau = dir_proj + 'tableau_data/'
if not os.path.exists(dir_tableau):
    os.makedirs(dir_tableau)

dir_pdf = './pdf/'
dir_tables = './tables/'
dir_table_pages = [dir_tables + d for d in sorted(os.listdir(dir_tables)) if not d.startswith('.')]

files = []

for dir_page in dir_table_pages:
    tmp_files = [dir_page + '/' + f for f in os.listdir(dir_page)]
    files.extend(tmp_files)

In [3]:
dir_pdf_pages = [dir_pdf + d for d in sorted(os.listdir(dir_pdf)) if not d.startswith('.')]
files_pdf = []
for dir_page in dir_pdf_pages:
    tmp_files = [dir_page + '/' + f for f in os.listdir(dir_page)]
    files_pdf.extend(tmp_files)

len(files_pdf)

4589

In [4]:
raw_imported = pd.read_csv('imported.tsv', sep='\t')
raw_fish_meat = pd.read_csv('fish_meat.tsv', sep='\t')
raw_foreign = pd.read_csv('foreign.tsv', sep='\t')

In [5]:
raw_imported.loc[(~raw_imported['제조일자(유통기한)'].str.startswith('2'))
                 &(~raw_imported['제조일자(유통기한)'].str.startswith('1'))
                 &(~raw_imported['제조일자(유통기한)'].str.startswith('('))]
# raw_imported.loc[raw_imported['제조일자(유통기한)']=='-'].shape

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
5080,당류가공품,아이치현,9999-12-31,201.6,불검출,불검출,적합,./tables/367/20141103_방사능_검사_현황-page-3-table-1.csv
51322,커피/건조,라오스,-,600,불검출,불검출,적합,./tables/288/20151113-page-3-table-1.csv
51323,커피/건조,페루,-,552,불검출,불검출,적합,./tables/288/20151113-page-3-table-1.csv
51324,커피/건조,에쿠아도르,-,450,불검출,불검출,적합,./tables/288/20151113-page-3-table-1.csv
59397,쇼트닝,효고현,`,75,불검출,불검출,적합,./tables/315/20150709-page-4-table-1.csv
67197,청주,교토부,청주,54,불검출,불검출,적합,./tables/343/20150302-page-4-table-1.csv
67198,자당지방산에스테르,미에현,혼합제제,750,불검출,불검출,적합,./tables/343/20150302-page-4-table-1.csv
67199,복합조미식품,에히메현,자당지방산\n에스테르,"2,880",불검출,불검출,적합,./tables/343/20150302-page-4-table-1.csv
67200,기타어육가공품,훗카이도,수산물조림,228,불검출,불검출,적합,./tables/343/20150302-page-4-table-1.csv
67368,혼합제제,사이타마현,혼합제제,70,불검출,불검출,적합,./tables/343/20150226-page-5-table-1.csv


In [6]:
raw_imported.info()
raw_imported.head(2)
# print(raw_imported.loc[raw_imported['제조일자(유통기한)'].str.contains('\(')].shape)
# raw_imported.loc[raw_imported['제조일자(유통기한)'].str.contains('\(')].head()
# idx_circul = raw_imported['제조일자(유통기한)'].str.contains('\(')
# raw_imported.loc[idx_circul, '제조일자(유통기한)'].str.extract(r'(\d{4}\-\d{1:2}\-\d{1:2})')
# raw_imported.loc[50956]
# raw_imported.loc[idx_circul, '제조일자(유통기한)'] = raw_imported.loc[idx_circul, '제조일자(유통기한)'].str.extract(r'(\d\d\d\d\-\d\d\-\d\d)')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138943 entries, 0 to 138942
Data columns (total 8 columns):
품목명           138943 non-null object
제조·생산지        138943 non-null object
제조일자(유통기한)    138943 non-null object
수입중량(kg)      138943 non-null object
세슘            138943 non-null object
요오드           138943 non-null object
판정            138943 non-null object
파일            138943 non-null object
dtypes: object(8)
memory usage: 8.5+ MB


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
0,당류가공품,에히메현,2014-10-09,161,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
1,청주,효고현,2014-11-04,114,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv


In [7]:
raw_imported.loc[raw_imported['제조·생산지'].str[0]=='2']

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
40292,소스류,2016-04-05,2016-03-01,"2,030.4",불검출,불검출,적합,./tables/256/20160421-page-5-table-1.csv
40293,혼합제제,2016-03-29,2016-03-01,"1,000",불검출,불검출,적합,./tables/256/20160421-page-5-table-1.csv
40294,청주,2016-03-10,2016-03-31,648,불검출,불검출,적합,./tables/256/20160421-page-5-table-1.csv
40295,청주,2016-03-01,2016-03-24,"1,296",불검출,불검출,적합,./tables/256/20160421-page-5-table-1.csv
40296,과자(스낵과자),2016-03-22,2016-03-10,288,불검출,불검출,적합,./tables/256/20160421-page-4-table-1.csv
40297,혼합제제,2016-04-01,2016-03-10,17,불검출,불검출,적합,./tables/256/20160421-page-4-table-1.csv
40298,혼합제제,2016-03-30,2016-03-10,200,불검출,불검출,적합,./tables/256/20160421-page-4-table-1.csv
40299,과자(쿠키),2016-03-22,2016-03-11,396,불검출,불검출,적합,./tables/256/20160421-page-4-table-1.csv
40300,과자(쿠키),2016-03-22,2016-04-05,600,불검출,불검출,적합,./tables/256/20160421-page-4-table-1.csv
40301,즉석조리식품,2016-03-17,2016-01-12,"3,678.49",불검출,불검출,적합,./tables/256/20160421-page-4-table-1.csv


In [8]:
raw_foreign.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
0,고형차,러시아연방,2022.07.31.,100,55,불검출,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
1,능이/건조,중국,(2019.08.01.),"2,230",3,-,적합,./tables/001/20190816+기타국가+방사능+검출+현황-page-1-table-1.csv
2,능이버섯/건조,중국,2020.10.31.,429.0,13,-,적합,./tables/002/201900813+기타국가+방사능+검출+현황-page-1-table-1.csv
3,잼,스웨덴,2020.10.27.,806,1,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv
4,차가버섯/건조,러시아연방,(2019.6.13.),"3,520",18,불검출,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv


In [9]:
tmp = raw_foreign['제조일자(유통기한)'].str.extract(r'(\d{4}[\-\.]?\d{0,2}[\-\.]?\d{0,2})')
tmp.head()
# tmp[0].str.replace('.', '-', regex=False)

,0
0,2022.07.31
1,2019.08.01
2,2020.10.31
3,2020.10.27
4,2019.6.13


# Preparation

## Functions

In [10]:
def fix_swapped(df):
    df = df.copy()
    idx_swapped = (df['제조·생산지'].str[0]=='2')&(df['제조일자(유통기한)'].str[0]!='2')
    df.loc[idx_swapped, ['제조·생산지','제조일자(유통기한)']] = df.loc[idx_swapped, ['제조일자(유통기한)','제조·생산지',]].values
    return df

In [11]:
def filter_9(df):
    df = df.copy()
    idx_not_9 = df['제조일자(유통기한)'].str[0]!='9'
    df = df.loc[idx_not_9]
    return df

In [12]:
def to_date(df):
    df = df.copy()
    df = df.loc[(df['제조일자(유통기한)'].str.startswith('2'))
               |(df['제조일자(유통기한)'].str.startswith('1'))
               |(df['제조일자(유통기한)'].str.startswith('('))]
    
    df['isProductionDate'] = True
    df.loc[(df['제조일자(유통기한)'].str.contains('\('))
           |(df['제조일자(유통기한)'].str.contains('\)')) , 'isProductionDate'] = False
    df.loc[~df['isProductionDate'], '제조일자(유통기한)'] = df.loc[~df['isProductionDate'], '제조일자(유통기한)'].str.extract(r'(\d{4}[\-\.]?\d{0,2}[\-\.]?\d{0,2})')
    df.loc[~df['isProductionDate'], '제조일자(유통기한)'] = df.loc[~df['isProductionDate'], '제조일자(유통기한)'].str.replace('.', '-', regex=False)
    df = df.loc[(df['제조일자(유통기한)']!='2019년 \n생산')]
    df = df.loc[(df['제조일자(유통기한)']!='2017년산')]
    
    df['제조일자(유통기한)'] = pd.to_datetime(df['제조일자(유통기한)'])
    return df

In [13]:
def to_date(df):
    df = df.copy()
    df = df.loc[(df['제조일자(유통기한)'].str.startswith('2'))
               |(df['제조일자(유통기한)'].str.startswith('1'))
               |(df['제조일자(유통기한)'].str.startswith('('))]
    
    df['isProductionDate'] = True
    df.loc[(df['제조일자(유통기한)'].str.contains('\('))
           |(df['제조일자(유통기한)'].str.contains('\)')) , 'isProductionDate'] = False
    df['제조일자(유통기한)'] = df['제조일자(유통기한)'].str.extract('(\d{4}[\-\.]?\d{0,2}[\-\.]?\d{0,2})')
    df['제조일자(유통기한)'] = df['제조일자(유통기한)'].str.replace('.', '-', regex=False)
    df = df.loc[(df['제조일자(유통기한)']!='2019년 \n생산')]
    df = df.loc[(df['제조일자(유통기한)']!='2017년산')]
    
    idx_tmp = df['제조일자(유통기한)']!='2019-0615'
    df = df.loc[idx_tmp]
    
    df['제조일자(유통기한)'] = pd.to_datetime(df['제조일자(유통기한)'])
    return df

In [14]:
df = raw_imported.copy()
df = fix_swapped(df)
df = filter_9(df)
# df = df.loc[(df['제조일자(유통기한)'].str.startswith('2'))
#            |(df['제조일자(유통기한)'].str.startswith('1'))
#            |(df['제조일자(유통기한)'].str.startswith('('))]

df['isProductionDate'] = True
df.loc[(df['제조일자(유통기한)'].str.contains('\('))
       |(df['제조일자(유통기한)'].str.contains('\)')) , 'isProductionDate'] = False
# df.loc[~df['isProductionDate'], '제조일자(유통기한)'] = df.loc[~df['isProductionDate'], '제조일자(유통기한)'].str.extract('(\d{4}[\-\.]?\d{0,2}[\-\.]?\d{0,2})')
df['제조일자(유통기한)'] = df['제조일자(유통기한)'].str.extract('(\d{4}[\-\.]?\d{0,2}[\-\.]?\d{0,2})')
df['제조일자(유통기한)'] = df['제조일자(유통기한)'].str.replace('.', '-', regex=False)
df = df.loc[(df['제조일자(유통기한)']!='2019년 \n생산')]
df = df.loc[(df['제조일자(유통기한)']!='2017년산')]

idx_tmp = df['제조일자(유통기한)']!='2019-0615'
df = df.loc[idx_tmp]

df['제조일자(유통기한)'] = pd.to_datetime(df['제조일자(유통기한)'])
df.loc[~df['isProductionDate']]

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
43666,과자(비스킷),아이치현,2017-02-02,345.6,불검출,불검출,적합,./tables/266/20160303-page-2-table-1.csv,False
43985,과자(비스킷),아이치현,2017-02-05,113.4,불검출,불검출,적합,./tables/266/20160302-page-2-table-1.csv,False
43986,과자(비스킷),아이치현,2017-02-05,113.4,불검출,불검출,적합,./tables/266/20160302-page-2-table-1.csv,False
43987,과자(웨이퍼),니가타현,2016-09-21,115.56,불검출,불검출,적합,./tables/266/20160302-page-2-table-1.csv,False
44398,빵류(그외 빵류),나가노현,2017-02-05,348.3,불검출,불검출,적합,./tables/267/20160229-page-1-table-2.csv,False
44399,과자(크래커),니가타현,2016-09-29,203.04,불검출,불검출,적합,./tables/267/20160229-page-1-table-2.csv,False
45067,기타가공품,도쿄도,2017-10-02,3,불검출,불검출,적합,./tables/269/20160217-page-1-table-2.csv,False
46558,기타코코아가공품,사이타마현,2016-07-01,79.36,불검출,불검출,적합,./tables/273/20160125-page-1-table-2.csv,False
46559,밀크초콜릿,사이타마현,2016-09-01,112.64,불검출,불검출,적합,./tables/273/20160125-page-1-table-2.csv,False
46766,초콜릿가공품,이바라키현,2016-10-01,912,불검출,불검출,적합,./tables/274/20160122-page-2-table-1.csv,False


In [15]:
def to_float(df):
    df = df.copy()
    df['수입중량(kg)'] = df['수입중량(kg)'].str.replace(',','')
    df['수입중량(kg)'] = df['수입중량(kg)'].str.replace('\n','')
    df = df.loc[df['수입중량(kg)']!='######']
    df['수입중량(kg)'] = df['수입중량(kg)'].astype(float)
    return df

In [16]:
def strip_whitespace(df):
    df = df.copy()
    df['품목명'] = df['품목명'].str.replace('\s+', '', regex=True)
    df['품목명'] = df['품목명'].str.replace('\n', '',)
    return df

In [17]:
def to_float_cesium_iodin(df):
    df = df.copy()
    zero_list = ['불검출**','불검출*','불검출','적합(1)','적합','-']
#     df = df.loc[~df['세슘'].isin()]
    for zero in zero_list:
        df['세슘'] = df['세슘'].str.replace(zero, '0', regex=False)
        df['요오드'] = df['요오드'].str.replace(zero, '0', regex=False)

    df['세슘'] = df['세슘'].str.replace(' Bq/kg', '', regex=False)
    df['요오드'] = df['요오드'].str.replace(' Bq/kg', '', regex=False)
    df['세슘'] = df['세슘'].astype(float)
    df['요오드'] = df['요오드'].astype(float)
    
    return df

In [18]:
tmp = raw_foreign.copy()
# zero_list = ['불검출','적합','적합(1)','불검출*','불검출**','-']

# for zero in zero_list[:5]:
tmp['세슘'] = tmp['세슘'].str.replace('불검출**', '0', regex=False)
#     tmp['요오드'] = tmp['요오드'].str.replace(zero, '0')

In [19]:
raw_imported.head(2)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
0,당류가공품,에히메현,2014-10-09,161,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv
1,청주,효고현,2014-11-04,114,불검출,불검출,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv


In [20]:
raw_imported.loc[~raw_imported['요오드'].str.isnumeric(), '요오드'].value_counts()

불검출    138943
Name: 요오드, dtype: int64

In [21]:
raw_fish_meat.loc[~raw_fish_meat['요오드'].str.isnumeric(), '요오드'].value_counts()

불검출      40487
적합       3217 
불검출**    1    
Name: 요오드, dtype: int64

In [22]:
raw_foreign.loc[~raw_foreign['요오드'].str.isnumeric(), '요오드'].value_counts()

불검출    402
-      107
Name: 요오드, dtype: int64

In [23]:
tmp = raw_foreign.copy()
tmp['세슘'] = tmp['세슘'].str.replace(' Bq/kg', '')
tmp['세슘'] = tmp['세슘'].str.replace(' ', '')
tmp['세슘'] = tmp['세슘'].astype(float)
tmp['세슘'].value_counts()
# tmp.loc[~tmp['세슘'].str.isdigit(), '세슘'].value_counts()

2.0      85
3.0      66
1.0      59
4.0      43
5.0      16
12.0     11
8.0      9 
17.0     9 
7.0      9 
6.0      8 
13.0     7 
19.0     6 
10.0     6 
44.0     6 
45.0     6 
33.0     6 
16.0     6 
35.0     5 
53.0     5 
49.0     5 
11.0     5 
47.0     5 
30.0     5 
9.0      5 
18.0     5 
48.0     5 
27.0     4 
15.0     4 
14.0     4 
59.0     3 
54.0     3 
71.0     3 
22.0     3 
40.0     3 
50.0     3 
43.0     3 
34.0     3 
70.0     3 
60.0     3 
36.0     3 
24.0     3 
37.0     3 
20.0     3 
26.0     2 
67.0     2 
66.0     2 
31.0     2 
52.0     2 
55.0     2 
65.0     2 
56.0     2 
42.0     2 
39.0     2 
41.0     2 
25.0     2 
58.0     2 
90.0     2 
29.0     2 
64.0     1 
78.0     1 
98.0     1 
219.0    1 
32.0     1 
81.0     1 
21.0     1 
265.0    1 
798.0    1 
62.0     1 
63.0     1 
61.0     1 
83.0     1 
120.0    1 
114.0    1 
28.0     1 
82.0     1 
57.0     1 
38.0     1 
73.0     1 
18.9     1 
8.8      1 
Name: 세슘, dtype: int64

In [24]:
def process_df(df):
    df = df.copy()
    df = fix_swapped(df)
    df = filter_9(df)
    df = to_date(df)
    df = to_float(df)
    df = strip_whitespace(df)
    df = to_float_cesium_iodin(df)
    return df

## Execution

In [25]:
print(raw_imported.shape)
imported = process_df(raw_imported)
print(imported.shape)
print(imported.info())

(138943, 8)
(138914, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 138914 entries, 0 to 138942
Data columns (total 9 columns):
품목명                 138914 non-null object
제조·생산지              138914 non-null object
제조일자(유통기한)          138913 non-null datetime64[ns]
수입중량(kg)            138914 non-null float64
세슘                  138914 non-null float64
요오드                 138914 non-null float64
판정                  138914 non-null object
파일                  138914 non-null object
isProductionDate    138914 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(3), object(4)
memory usage: 9.7+ MB
None


In [26]:
print(raw_fish_meat.shape)
fish_meat = process_df(raw_fish_meat)
print(fish_meat.shape)
print(fish_meat.info())

(43705, 8)
(43705, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 43705 entries, 0 to 43704
Data columns (total 9 columns):
품목명                 43705 non-null object
제조·생산지              43705 non-null object
제조일자(유통기한)          43705 non-null datetime64[ns]
수입중량(kg)            43705 non-null float64
세슘                  43705 non-null float64
요오드                 43705 non-null float64
판정                  43705 non-null object
파일                  43705 non-null object
isProductionDate    43705 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(3), object(4)
memory usage: 3.0+ MB
None


In [27]:
print(raw_foreign.shape)
foreign = process_df(raw_foreign)
print(foreign.shape)
print(foreign.info())

(509, 8)
(493, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 493 entries, 0 to 508
Data columns (total 9 columns):
품목명                 493 non-null object
제조·생산지              493 non-null object
제조일자(유통기한)          493 non-null datetime64[ns]
수입중량(kg)            493 non-null float64
세슘                  493 non-null float64
요오드                 493 non-null float64
판정                  493 non-null object
파일                  493 non-null object
isProductionDate    493 non-null bool
dtypes: bool(1), datetime64[ns](1), float64(3), object(4)
memory usage: 35.1+ KB
None


In [91]:
raw_foreign.loc[raw_foreign['제조일자(유통기한)']=='2019년 \n생산']
raw_foreign.loc[raw_foreign['제조일자(유통기한)']=='2017년산']

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일
405,냉동능이버섯,중국,2017년산,"10,330",2,불검출,적합,./tables/145/cleaned_20171116_기타국가_방사능_검출_현황(중국산+능이버섯)-page-1-table-1.csv
418,송이버섯,중국,2017년산,88.0,1,불검출,적합,./tables/150/cleaned_20171023_기타국가_방사능_검출_현황(중국산+송이버섯)-page-1-table-1.csv
425,건 차가버섯,러시아연방,2017년산,"3,000",27,불검출,적합,./tables/160/cleaned_20170828_기타국가_방사능_검출_현황(러시아산+차가버섯)-page-1-table-1.csv
426,능이버섯,중국,2017년산,104.0,5,불검출,적합,./tables/160/cleaned_20170824_기타국가_방사능_검출_현황(중국산+능이버섯)-page-1-table-1.csv


## Japan States

In [33]:
tmp = imported['제조·생산지'].value_counts().to_frame()
japan_imported = tmp.loc[(tmp['제조·생산지']>300) & (tmp.index!='ETC')].index.tolist()

fish_states = raw_fish_meat['제조·생산지'].value_counts().index.tolist()

set(fish_states) - set(japan_imported)

tmp = 'KAGAWA, 가가와, 가고시마, 가나가와, 가이아나, 구마모토, 나가사키, 도쿄, 도토리, 도토리현, 돗토리, 미에, 사이타마, 에이메현, 에미헤현, 야마카타현, 오사카, 카가와, 홋카이도, 후쿠야마, 후쿠오카, 후쿠오카부'
japan_fish_exclusive = tmp.split(', ')
# print(japan_fish_exclusive)

set(fish_states) - set(japan_imported)

foreign_states = raw_foreign['제조·생산지'].value_counts().index.tolist()

set(foreign_states) - set(japan_imported)

japan_states = japan_imported + japan_fish_exclusive
japan_states

['효고현',
 '아이치현',
 '오사카부',
 '사이타마현',
 '니가타현',
 '후쿠오카현',
 '히로시마현',
 '훗카이도',
 '시즈오카현',
 '교토부',
 '나가노현',
 '미에현',
 '가나가와현',
 '지바현',
 '이바라키현',
 '도치기현',
 '오카야마현',
 '시가현',
 '도쿄도',
 '사가현',
 '에히메현',
 '나라현',
 '구마모토현',
 '나가사키현',
 '군마현',
 '오이타현',
 '와카야마현',
 '시마네현',
 '가고시마현',
 '야마가타현',
 '기후현',
 '야마나시현',
 '돗토리현',
 '후쿠시마현',
 '야마구치현',
 '도쿠시마현',
 '고치현',
 '이와테현',
 '미야자키현',
 '아키타현',
 '도야마현',
 '가가와현',
 '아오모리현',
 '후쿠이현',
 '오키나와현',
 '이시카와현',
 '미야기현',
 'KAGAWA',
 '가가와',
 '가고시마',
 '가나가와',
 '가이아나',
 '구마모토',
 '나가사키',
 '도쿄',
 '도토리',
 '도토리현',
 '돗토리',
 '미에',
 '사이타마',
 '에이메현',
 '에미헤현',
 '야마카타현',
 '오사카',
 '카가와',
 '홋카이도',
 '후쿠야마',
 '후쿠오카',
 '후쿠오카부']

## Not Japan

In [34]:
# not japan from general
tmp = imported['제조·생산지'].value_counts().to_frame()
not_japan = tmp.index.tolist()
not_japan = set(not_japan) - set(japan_states)
misclassified = set(['건어포류', '고형차', '곡류가공품', '과자(비스킷)', '과자(스낵과자)', '과자(크래커)', '국수', '금박', '기타가공품', '기타식용유지', '당류가공품',
                 '드레싱', '맥주', '몰식자산', '발효식초', '백설탕', '복합조미식품', '볶은커피', '소스류', '수산물가공품', '액상차', '장류절임', '전분가공품',
                 '조미된장', '즉석조리식품', '청주', '초콜릿가공품', '추잉껌', '칼슘', '캔디류(사탕)', '향신료조제품(고\n추냉이가공품)', '혼합제제'])
not_japan = not_japan - set(misclassified)
not_japan

{'2015-03-11',
 '2015-11-24',
 '2015-12-02',
 '2015-12-31',
 '2016-01-02',
 '2016-01-16',
 '2016-01-19',
 '2016-01-20',
 '2016-01-26',
 '2016-01-31',
 '2016-02-02',
 '2016-02-08',
 '2016-02-10',
 '2016-02-20',
 '2016-02-22',
 '2016-02-24',
 '2016-02-26',
 '2016-02-29',
 '2016-03-01',
 '2016-03-02',
 '2016-03-07',
 '2016-03-09',
 '2016-03-10',
 '2016-03-11',
 '2016-03-12',
 '2016-03-14',
 '2016-03-15',
 '2016-03-16',
 '2016-03-17',
 '2016-03-18',
 '2016-03-19',
 '2016-03-21',
 '2016-03-22',
 '2016-03-23',
 '2016-03-24',
 '2016-03-25',
 '2016-03-26',
 '2016-03-28',
 '2016-03-29',
 '2016-03-30',
 '2016-03-31',
 '2016-04-01',
 '2016-04-02',
 '2016-04-04',
 '2016-04-05',
 '2016-04-06',
 '2016-04-07',
 '2016-04-11',
 'ETC',
 '가나',
 '가나\n(수출국:일본)',
 '과테말라',
 '남 아 프 리 카\n공화국',
 '네덜란드',
 '네덜란드\n(수출국:일본)',
 '뉴질랜드',
 '니카라과',
 '니카라과\n(수출국: 일본)',
 '대한민국',
 '덴마크',
 '독일',
 '독일\n(수출국: 일본)',
 '라오스',
 '라오스\n(수출국:일본)',
 '러시아 연방',
 '루안다',
 '르완다',
 '멕시코',
 '미국',
 '미국\n(수출국: 일본)',
 '베트남',
 '벨기에',
 '벨기에\n(수출

In [35]:
# not japan from fish
tmp = fish_meat['제조·생산지'].value_counts().to_frame()
not_japan_fish = tmp.index.tolist()
not_japan_fish = set(not_japan_fish) - set(japan_states)
# misclassified = set(['건어포류', '고형차', '곡류가공품', '과자(비스킷)', '과자(스낵과자)', '과자(크래커)', '국수', '금박', '기타가공품', '기타식용유지', '당류가공품',
#                  '드레싱', '맥주', '몰식자산', '발효식초', '백설탕', '복합조미식품', '볶은커피', '소스류', '수산물가공품', '액상차', '장류절임', '전분가공품',
#                  '조미된장', '즉석조리식품', '청주', '초콜릿가공품', '추잉껌', '칼슘', '캔디류(사탕)', '향신료조제품(고\n추냉이가공품)', '혼합제제'])
# not_japan_fish = not_japan_fish - set(misclassified)
not_japan_fish

{'가나',
 '감비아',
 '그루지야',
 '그리스',
 '그린랜드',
 '기니',
 '기니비사우',
 '나미비아',
 '남아프리카공화국',
 '네덜란드',
 '노르웨이',
 '뉴질랜드',
 '니카라과',
 '대만',
 '대서양',
 '덴마크',
 '라오스',
 '라이베리아',
 '라트비아',
 '러시아',
 '루마니아',
 '리비아',
 '리투아니아',
 '마다가스카르',
 '마샬',
 '마이크로\n네시아',
 '마이크로네\n시아',
 '마이크로네시아',
 '말레이시아',
 '멕시코',
 '모로코',
 '모리셔스',
 '모리타니',
 '모잠비크',
 '몰디브',
 '몰타',
 '미국',
 '미얀마',
 '바누아투',
 '바레인',
 '방글라데시',
 '베네수엘라',
 '베트남',
 '벨리즈',
 '불가리아',
 '브라질',
 '사가',
 '사우디아라\n비아',
 '사우디아라비아',
 '사이프러스',
 '세네갈',
 '세이셸',
 '세인트빈센트그레\n나딘',
 '소말리아',
 '솔로몬\n군도',
 '솔로몬군도',
 '수리남',
 '스리랑카',
 '스페인',
 '시리아',
 '시에라리온',
 '싱가포르',
 '아랍에미리\n트연합',
 '아랍에미리트연합',
 '아르헨티나',
 '아이슬란드',
 '아일랜드',
 '알제리',
 '앙골라',
 '에스토니아',
 '에콰도르',
 '영국',
 '예멘',
 '오만',
 '온두라스',
 '우루과이',
 '우크라이나',
 '이란',
 '이집트',
 '이탈리아',
 '인도',
 '인도네시아',
 '인도양',
 '일본',
 '중국',
 '중국\n홍콩(\n)',
 '중국 홍콩(\n)',
 '중국(홍콩)',
 '칠레',
 '캄보디아',
 '캐나다',
 '케냐',
 '코스타리카',
 '코트디부아르',
 '콜롬비아',
 '쿠바',
 '크로아티아',
 '키리바시',
 '탄자니아',
 '태국',
 '태평양',
 '터키',
 '통가',
 '튀니지',
 '파나마',
 '파키스탄',
 '파퓨아\n뉴기니',
 '파퓨아뉴기니',
 '페로제도',
 '

## Branch DFs

일반식품, 축수산물  
일본, 일본외  
경우의 수 4가지

In [36]:
idx_general_japan = imported['제조·생산지'].isin(japan_states)
print(imported.shape)
print(imported.loc[idx_general_japan].shape)
imported.head()

general_japan = imported.loc[idx_general_japan]
general_japan.head()

general_japan['세슘'].value_counts()

(138914, 9)
(137400, 9)


0.0     137385
1.0     5     
2.0     4     
16.0    1     
8.0     1     
28.0    1     
4.0     1     
26.0    1     
3.0     1     
Name: 세슘, dtype: int64

In [37]:
general_not_japan = imported.loc[imported['제조·생산지'].isin(not_japan)]
general_not_japan = general_not_japan.append(foreign)
print(general_not_japan.shape)
general_not_japan.head()

(1935, 9)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
141,커피/건조,브라질,2012-09-06,6000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True
151,커피/건조,브라질,2014-06-26,12000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True
1298,육두구(육두관)/건조,인도네시아,2014-08-01,3000.0,0.0,0.0,적합,./tables/355/20141226_방사능_검사_현황-page-1-table-2.csv,True
1340,"회향(소회향,판넬,FoeniculumvulgareMill)/건조",인도,2014-10-01,2000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-3-table-1.csv,True
1481,커피/건조,브라질,2014-06-26,6000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-2-table-1.csv,True


In [38]:
fish_japan = fish_meat.loc[fish_meat['제조·생산지'].isin(japan_states)]
print(fish_japan.shape)
fish_japan.sample(5)

(34663, 9)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
21081,냉장명태,홋카이도,2019-06-19,1056.0,0.0,0.0,적합,./tables/015/일본산+및+기타국가산+수입수산물+방사능검사+현황(6월+19일+완료건)-page-1-table-1.csv,True
122,냉장명태,홋카이도,2017-03-10,3000.0,0.0,0.0,적합,./tables/186/일본산_수입축수산물_방사능검사_현황(3월10일_완료건)-page-2-table-1.csv,True
2879,냉장가오리,에히메현,2016-11-25,630.0,0.0,0.0,적합,./tables/208/일본산_수입축수산물_방사능검사_현황(11월25일_완료건)-page-1-table-1.csv,True
27961,활가리비,홋카이도,2018-07-13,7300.0,0.0,0.0,적합,./tables/091/일본산+및+기타국가산+수입수산물+방사능검사+현황(7월13일+완료건)-page-1-table-1.csv,True
37892,냉장돔(황돔),후쿠오카현,2017-05-18,120.0,0.0,0.0,적합,./tables/175/일본산+및+기타국가산+수입수산물+방사능검사+현황(5월18일+완료건)-page-1-table-1.csv,True


In [39]:
fish_not_japan = fish_meat.loc[fish_meat['제조·생산지'].isin(not_japan_fish)]
print(fish_not_japan.shape)
fish_not_japan.sample(5)

(9042, 9)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
36848,냉동기름치,대서양,2017-07-06,7337.0,0.0,0.0,적합,./tables/169/일본산+및+기타국가산+수입수산물+방사능검사+현황(7월6일+완료건)-page-1-table-1.csv,True
31782,"냉동다랑어(눈다랑어,F,횟감)",태평양,2018-01-23,30.0,0.0,0.0,적합,./tables/129/일본산+및+기타국가산+수입수산물+방사능검사+현황(1월23일+완료건)-page-1-table-1.csv,True
38166,냉동곤이(명태),러시아,2017-05-08,17955.0,0.0,0.0,적합,./tables/177/일본산+및+기타국가산+수입수산물+방사능검사+현황(5월8일+완료건)-page-2-table-1.csv,True
25479,냉동가자미(각시가자미),러시아,2018-11-26,22050.0,0.0,0.0,적합,./tables/061/일본산+및+기타국가산+수입수산물+방사능검사+현황(11월26일+완료건)-page-2-table-1.csv,True
22272,냉동명태(F),러시아,2019-04-09,21200.0,0.0,0.0,적합,./tables/033/일본산+및+기타국가산+수입수산물+방사능검사+현황(4월+9일+완료건)-page-2-table-2.csv,True


In [40]:
foreign.head()
foreign['품목명'].value_counts()

고형차               88
잼                 66
능이/냉동             62
능이/건조             57
과채가공품             40
냉동능이버섯            21
건조능이버섯            15
과․채가공품            14
능이버섯              10
차가버섯              10
액상차               7 
그물버섯              6 
블루베리              6 
빌베리추출물(원료성)       6 
천연향신료             4 
건조그물버섯            4 
꾀꼬리버섯             4 
송이버섯              4 
차가버섯/건조           3 
링곤베리잼             3 
링곤베리드링크컨센트레이트     3 
침출차               3 
건차가버섯             3 
기타잼               3 
잼류                3 
기타가공품             2 
곰보버섯              2 
건조차가버섯            2 
능이버섯/건조           2 
냉동월귤나무열매          2 
빌베리추출물            2 
유기농링곤베리잼          2 
냉동뿔나팔버섯           1 
쨈                 1 
건조그물버섯(농산물)       1 
냉동능이              1 
과.채가공품            1 
과․채음료             1 
개암(헤즐너트)/탈각/건조    1 
냉동꾀꼬리버섯           1 
천연향신료(건조제품)       1 
유기농블루베리잼          1 
시나몬스틱             1 
블루베리/냉동           1 
냉동송이버섯            1 
블루베리/열매/냉동        1 
블루베리잼(잼)          1 
링곤베리잼(잼류)    

## 제조·생산지 표준화

In [41]:
def normalize_location(df):
    df = df.copy()
    df['제조·생산지'] = df['제조·생산지'].str.replace(r'\s+', '')
    df['제조·생산지'] = df['제조·생산지'].str.replace(r'\(.+\)', '')
    df['제조·생산지'] = df['제조·생산지'].str.replace('^중구$', '중국')
    df['제조·생산지'] = df['제조·생산지'].str.replace('러시아연방', '러시아', regex=False)
    df['제조·생산지'] = df['제조·생산지'].str.replace('산동성', '중국', regex=False)
    df['제조·생산지'] = df['제조·생산지'].str.replace('KAGAWA', '가가와현', regex=False)
    df['제조·생산지'] = df['제조·생산지'].str.replace('카가와', '가가와현', regex=False)
    df['제조·생산지'] = df['제조·생산지'].str.replace('에미헤현', '에히메현', regex=False)
    df['제조·생산지'] = df['제조·생산지'].str.replace('에이메현', '에히메현', regex=False)
    return df

In [42]:
def extract_inspection_date(df):
    df = df.copy()
    df['inspection_date'] = df['파일'].str.extract('.+\/\d{3}\/.*?(\d{4,9})')
    idx_date_4 = df['inspection_date'].str.len()==4
    df.loc[idx_date_4, 'inspection_date'] = df.loc[idx_date_4, '제조일자(유통기한)'].dt.year.astype(str) + df.loc[idx_date_4, 'inspection_date']
    df['inspection_date'] = df['inspection_date'].str.replace('201900813', '20190813')
    df['inspection_date'] = pd.to_datetime(df['inspection_date'])
    return df


def extract_inspection_date_fish(df):
    df = df.copy()
    df['inspection_date'] = df['제조일자(유통기한)']
    return df


def process_df_2(df, general=True):
    df = normalize_location(df)
    df = extract_inspection_date(df) if general else extract_inspection_date_fish(df)
    return df


gen_not_japan = process_df_2(general_not_japan)
# gen_not_japan['제조·생산지'].value_counts()
gen_not_japan.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
141,커피/건조,브라질,2012-09-06,6000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True,2015-01-15
151,커피/건조,브라질,2014-06-26,12000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True,2015-01-15
1298,육두구(육두관)/건조,인도네시아,2014-08-01,3000.0,0.0,0.0,적합,./tables/355/20141226_방사능_검사_현황-page-1-table-2.csv,True,2014-12-26
1340,"회향(소회향,판넬,FoeniculumvulgareMill)/건조",인도,2014-10-01,2000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-3-table-1.csv,True,2014-12-24
1481,커피/건조,브라질,2014-06-26,6000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-2-table-1.csv,True,2014-12-24


In [43]:
gen_japan = process_df_2(general_japan)
# gen_japan['제조·생산지'].value_counts().to_frame().reset_index().sort_values('index')
gen_japan.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
0,당류가공품,에히메현,2014-10-09,161.0,0.0,0.0,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv,True,2015-01-14
1,청주,효고현,2014-11-04,114.0,0.0,0.0,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv,True,2015-01-14
2,청주,효고현,2014-09-01,95.0,0.0,0.0,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv,True,2015-01-14
3,발효식초,도쿄도,2014-11-12,144.0,0.0,0.0,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv,True,2015-01-14
4,청주,효고현,2014-11-13,281.0,0.0,0.0,적합,./tables/352/20150114_방사능_검사_현황-page-1-table-2.csv,True,2015-01-14


In [44]:
fsh_japan = process_df_2(fish_japan, general=False)
# fsh_japan['제조·생산지'].value_counts().to_frame().reset_index().sort_values('index')
fsh_japan.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
0,활돔(참돔),에히메현,2017-03-17,4672.0,0.0,0.0,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv,True,2017-03-17
1,냉장명태,홋카이도,2017-03-17,1157.0,0.0,0.0,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv,True,2017-03-17
2,냉장명태,홋카이도,2017-03-17,1280.0,0.0,0.0,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv,True,2017-03-17
3,냉장대구,홋카이도,2017-03-17,490.0,0.0,0.0,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv,True,2017-03-17
4,냉장명태,홋카이도,2017-03-17,1144.0,0.0,0.0,적합,./tables/184/일본산_수입축수산물_방사능검사_현황(3월17일_완료건)-page-1-table-1.csv,True,2017-03-17


In [45]:
fsh_not_japan = process_df_2(fish_not_japan, general=False)
fsh_not_japan['제조·생산지'].value_counts().to_frame().reset_index().sort_values('index')
# fsh_not_japan.head()

,index,제조·생산지
71,가나,4
46,감비아,5
100,그루지야,1
75,그리스,4
35,그린랜드,7
42,기니,6
68,기니비사우,4
55,나미비아,5
38,남아프리카공화국,7
51,네덜란드,5


In [46]:
# df = fish_japan
# df['inspection_date'] = df['파일'].str.extract('.+\/\d{3}\/.*?(\d{4,9})')
# df.loc[df['inspection_date'].isna()].head()

In [47]:
prohibit = ['후쿠시마', '이바라키', '군마', '미야기', '이와테', '도치기', '지바', '아오모리',]

In [48]:
foreign_from_general = imported.loc[imported['제조·생산지'].isin(not_japan)]
print(foreign_from_general.shape)
foreign_from_general.head()

(1442, 9)


,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
141,커피/건조,브라질,2012-09-06,6000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True
151,커피/건조,브라질,2014-06-26,12000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True
1298,육두구(육두관)/건조,인도네시아,2014-08-01,3000.0,0.0,0.0,적합,./tables/355/20141226_방사능_검사_현황-page-1-table-2.csv,True
1340,"회향(소회향,판넬,FoeniculumvulgareMill)/건조",인도,2014-10-01,2000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-3-table-1.csv,True
1481,커피/건조,브라질,2014-06-26,6000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-2-table-1.csv,True


# Analysis

## 국가별 검사 횟수 및 중량

In [168]:
gen_japan.head()
tmp = gen_japan['수입중량(kg)'].agg({'수입중량(kg)':['sum','count']}).to_frame().reset_index()
tmp.drop('level_0', axis=1, inplace=True)
tmp['country'] = '일본'
tmp = tmp.pivot(index='country', columns='level_1', values='수입중량(kg)').reset_index()
tmp.columns.name = ''
tmp

/home/jake/.virtualenvs/workspace/lib/python3.6/site-packages/IPython/kernel/__main__.py:2: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  from ipykernel import kernelapp as app


,country,count,sum
0,일본,137400.0,5.680975e+08


In [171]:
gen_not_japan.head()
tmp2 = gen_not_japan.loc[(gen_not_japan['제조·생산지'].str[0]!='2')
                       & (gen_not_japan['제조·생산지']!='ETC')].groupby('제조·생산지').agg({'수입중량(kg)':['sum','count']}).reset_index()
# tmp2.columns = tmp2.columns.droplevel(0)
tmp2.columns = ['country', 'sum','count']
tmp2 = tmp2.sort_values('count', ascending=False).head(10)

In [174]:
tmp3 = tmp2.append(tmp).sort_values('count', ascending=False)
tmp3.columns = ['검사횟수','국가','검사중량(kg)']
tmp3

,검사횟수,국가,검사중량(kg)
0,137400.0,일본,5.680975e+08
20,208.0,브라질,2.825936e+06
39,207.0,중국,1.451836e+06
10,111.0,러시아,1.465602e+05
16,69.0,미국,2.605427e+05
46,66.0,콜롬비아,9.770369e+05
60,55.0,프랑스,5.012669e+04
1,43.0,과테말라,3.114774e+05
18,42.0,벨기에,3.908900e+03
22,35.0,스웨덴,7.005120e+04


### Export: 국가별 검사 횟수 및 중량

In [175]:
# tmp3.to_csv(dir_tableau + 'inspection_by_country.tsv', sep='\t', index=False)

## 일본 식품 방사능 검출 X 판정

In [190]:
gen_japan.head()
# gen_japan['판정'].value_counts()
tmp = gen_japan.loc[gen_japan['세슘']!=0].copy()
tmp['판정'] = tmp['판정'].str.replace('취하*', '취하', regex=False)
# tmp.groupby('수입중량(kg)').agg({'판정':'count'})
# tmp.head()
# pd.crosstab(tmp['수입중량(kg)'], tmp['판정'], margins=True).style.background_gradient(cmap='summer_r')
tmp = pd.crosstab(tmp['세슘'], tmp['판정'], margins=True).reset_index()
tmp.drop('All', axis=1, inplace=True)
tmp.drop(8, inplace=True)
tmp
# tmp.columns

판정,세슘,적합,취하
0,1,0,5
1,2,1,3
2,3,0,1
3,4,0,1
4,8,0,1
5,16,0,1
6,26,1,0
7,28,0,1


### Export: 일본 식품 방사능 검출 X 판정

In [191]:
# tmp.to_csv(dir_tableau + 'decision_japan.tsv', sep='\t', index=False)

## 일본 식품 방사능 검출 추이

In [206]:
gen_japan.head()
tmp = gen_japan.loc[gen_japan['세슘']!=0]
tmp = tmp.groupby(tmp['inspection_date'].dt.year).agg({'세슘':'sum'}).reset_index()
tmp.columns = ['검사연도', '세슘검출량']
# tmp[['검사연도','세슘검출량']] = ['2015',0]
tmp = tmp.append({'검사연도':2015,'세슘검출량':0}, ignore_index=True)
tmp = tmp.append({'검사연도':2016,'세슘검출량':0}, ignore_index=True)
tmp = tmp.sort_values('검사연도', ascending=True)
tmp

,검사연도,세슘검출량
0,2014,28.0
4,2015,0.0
5,2016,0.0
1,2017,7.0
2,2018,49.0
3,2019,14.0


### Export: 일본 식품 방사능 검출 추이

In [207]:
# tmp.to_csv(dir_tableau + 'japan_trend.tsv', sep='\t', index=False)

## 품목별 수입 중량

가리비 수입량 높다 => 일본 내수 부진으로 한국에 수출, 기사 확인

In [49]:
fsh_japan.groupby('품목명').agg({'수입중량(kg)':'sum'}).sort_values('수입중량(kg)', ascending=False)

,수입중량(kg)
품목명,
활가리비,28769990.00
활우렁쉥이,17731783.00
냉장명태,17498800.00
활돔(참돔),10932506.00
냉장홍어,6640356.30
냉장갈치,4976191.91
활방어,3910576.00
냉동가리비(외투막),3346271.00
냉동멸치,2539760.00


In [50]:
gen_japan.loc[gen_japan['세슘']!=0].groupby('품목명').agg({'수입중량(kg)':'count'}).sort_values('수입중량(kg)', ascending=False)

,수입중량(kg)
품목명,
아연,2
침출차,2
고형차,1
기타가공품,1
기타수산물가공품,1
면류첨가알칼리제,1
볶은커피,1
볶은커피*,1
빌베리추출물(원료성),1


In [51]:
gen_japan.loc[gen_japan['세슘']!=0]

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
8829,볶은커피*,사이타마현,2014-05-30,23.04,2.0,0.0,적합,./tables/380/20140826_방사능_검사_현황-page-3-table-1.csv,True,2014-08-26
11436,아연*,오사카부,2014-01-25,171.75,26.0,0.0,적합,./tables/389/20140714_방사능_검사_현황-page-3-table-1.csv,True,2014-07-14
71186,천연향신료,오사카부,2019-04-04,60.00,3.0,0.0,취하,./tables/005/20190801+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2019-08-01
72995,혼합제제,시가현,2019-06-11,25.00,8.0,0.0,취하,./tables/011/20190704+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv,True,2019-07-04
82589,기타수산물가공품,에히메현,2019-01-24,5025.00,1.0,0.0,취하,./tables/038/20190314+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2019-03-14
83040,볶은커피,미에현,2019-01-02,42.00,2.0,0.0,취하,./tables/039/20190311+일본산+수입식품+검사현황+일일보고-page-7-table-1.csv,True,2019-03-11
94050,빌베리추출물(원료성),지바현,2018-03-01,40.00,2.0,0.0,취하,./tables/075/20180917+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2018-09-17
97461,아연,오사카부,2021-06-10,31.00,28.0,0.0,취하,./tables/086/20180730+일본산+수입식품+검사현황+일일보고-page-6-table-1.csv,False,2018-07-30
112302,침출차,가고시마현,2017-12-19,20.00,1.0,0.0,취하,./tables/129/20180123+일본산+수입식품+검사현황+일일보고-page-3-table-1.csv,True,2018-01-23
114216,고형차,시즈오카현,2017-11-01,15.00,1.0,0.0,취하,./tables/135/20171222+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2017-12-22


In [52]:
tmp = fsh_japan.groupby([fsh_japan['품목명'], fsh_japan['제조일자(유통기한)'].dt.year]).agg({'수입중량(kg)':'sum'}).reset_index()
tmp = tmp.pivot(index='품목명', columns='제조일자(유통기한)', values='수입중량(kg)')
tmp.fillna(0, inplace=True)
tmp['total'] = tmp.sum(axis=1)
tmp.sort_values('total', ascending=False)
# .sort_values('수입중량(kg)', ascending=False)

제조일자(유통기한),2014,2015,2016,2017,2018,2019,total
품목명,,,,,,,
활가리비,3567832.0,5462566.0,5584108.00,4926981.00,5880686.0,3347817.0,28769990.00
활우렁쉥이,1447064.0,2514379.0,3137832.00,5081023.00,4072947.0,1478538.0,17731783.00
냉장명태,1674137.0,3291388.0,3810303.00,3793697.00,3545126.0,1384149.0,17498800.00
활돔(참돔),1642193.0,1141814.0,1770825.00,2056333.00,2754090.0,1567251.0,10932506.00
냉장홍어,669338.0,1182902.0,1258503.30,1340517.00,1430879.0,758217.0,6640356.30
냉장갈치,795436.0,1353924.0,1314261.00,833143.91,379820.0,299607.0,4976191.91
활방어,85708.0,294467.0,474283.00,810594.00,1484038.0,761486.0,3910576.00
냉동가리비(외투막),785740.0,455975.0,759563.00,819393.00,293870.0,231730.0,3346271.00
냉동멸치,834090.0,163530.0,977585.00,437730.00,21750.0,105075.0,2539760.00


In [53]:
fsh_japan.groupby(fsh_japan['제조일자(유통기한)'].dt.year).agg({'수입중량(kg)':'sum'})

,수입중량(kg)
제조일자(유통기한),
2014,14442134.00
2015,20165388.60
2016,22514363.54
2017,23031199.45
2018,23266706.00
2019,11894717.00


## Samples

In [54]:
fish_japan.sample(5)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
18439,활우렁쉥이,홋카이도,2014-07-18,8525.0,0.0,0.0,적합,./tables/388/일본산_수입축수산물_방사능검사_현황(7.18일_완료건)-page-1-table-1.csv,True
14893,냉장갈치,야마구치현,2015-02-10,352.0,0.0,0.0,적합,./tables/346/일본산_수입축수산물_방사능검사_현황(2.10일_완료건)-page-1-table-1.csv,True
35453,활우렁쉥이,홋카이도,2017-09-20,2991.0,0.0,0.0,적합,./tables/156/일본산+및+기타국가산+수입수산물+방사능검사+현황(9월20일+완료건)-page-1-table-1.csv,True
17345,냉장명태,홋카이도,2014-10-08,3302.0,0.0,0.0,적합,./tables/372/일본산_수입축수산물_방사능검사_현황(10.8일_완료건)-page-1-table-1.csv,True
5463,냉장갈치,히로시마현,2016-07-14,95.0,0.0,0.0,적합,./tables/238/일본산_수입축수산물_방사능검사_현황(7월14일_완료건)-page-1-table-1.csv,True


In [55]:
general_japan.sample(5)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
68908,화이트초콜릿,훗카이도,2015-01-07,397.5,0.0,0.0,적합,./tables/349/20150126-page-5-table-1.csv,True
90902,초콜릿,훗카이도,2018-09-21,68.0,0.0,0.0,적합,./tables/067/20181029+일본산+수입식품+검사현황+일일보고-page-6-table-1.csv,True
51094,양조간장,아이치현,2015-10-08,1080.0,0.0,0.0,적합,./tables/288/20151113-page-2-table-1.csv,True
26039,혼합제제,효고현,2016-11-04,30.0,0.0,0.0,적합,./tables/209/20161122-page-3-table-1.csv,True
46384,청주,미야기현,2015-10-14,86.4,0.0,0.0,적합,./tables/273/20160125-page-2-table-1.csv,True


In [56]:
foreign.sample(5)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate
245,과채가공품,프랑스,2020-10-01,240.0,2.0,0.0,적합,./tables/088/20180724_기타국가_방사능_검출_현황-page-1-table-1.csv,True
276,고형차,러시아연방,2020-03-11,625.0,61.0,0.0,적합,./tables/100/20180604_기타국가_방사능_검출_현황-page-1-table-1.csv,True
146,액상차,러시아연방,2021-11-04,230.0,6.0,0.0,적합,./tables/056/20181214_기타국가_방사능_검출_현황-page-1-table-1.csv,True
461,블루베리잼,스웨덴,2017-11-23,856.8,3.0,0.0,적합,"./tables/220/20161011+기타국가+방사능+검출+현황(중국+능이,+스웨덴+베리류2건)-page-1-table-1.csv",True
142,능이/건조,중국,2021-06-02,1105.0,2.0,0.0,적합,./tables/055/20181219_기타국가_방사능_검출_현황-page-1-table-1.csv,True


## 연도별 수입량

In [57]:
fsh_japan.groupby(fish_japan['제조일자(유통기한)'].dt.year).agg({'수입중량(kg)':'sum'})

,수입중량(kg)
제조일자(유통기한),
2014,14442134.00
2015,20165388.60
2016,22514363.54
2017,23031199.45
2018,23266706.00
2019,11894717.00


In [58]:
gen_japan.groupby(general_japan['제조일자(유통기한)'].dt.year).agg({'수입중량(kg)':'sum'})

,수입중량(kg)
제조일자(유통기한),
2006.0,1.200000e+02
2007.0,6.342400e+02
2009.0,7.392000e+02
2010.0,9.072000e+02
2011.0,4.214800e+03
2012.0,1.833408e+05
2013.0,5.473062e+05
2014.0,5.910562e+07
2015.0,7.862756e+07


In [59]:
foreign.groupby(foreign['제조일자(유통기한)'].dt.year).agg({'수입중량(kg)':'sum'})

,수입중량(kg)
제조일자(유통기한),
2014,100.00
2016,48164.60
2017,188217.70
2018,319401.00
2019,624078.63
2020,179529.55
2021,126702.92
2022,40650.90


## 연도별 세슘, 요오드 검출량

In [60]:
tmp = gen_japan
tmp.groupby(tmp['inspection_date'].dt.year).agg({'세슘':'sum'})

,세슘
inspection_date,
2014,28.0
2015,0.0
2016,0.0
2017,7.0
2018,49.0
2019,14.0


In [123]:
gen_japan['세슘'].sum()

98.0

In [61]:
gen_japan.loc[gen_japan['세슘']!=0].groupby(gen_japan['inspection_date'].dt.year).agg({'세슘':'count'})

,세슘
inspection_date,
2014,2
2017,4
2018,5
2019,4


In [62]:
gen_japan.loc[gen_japan['세슘']!=0].sort_values('inspection_date', ascending=False)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
71186,천연향신료,오사카부,2019-04-04,60.00,3.0,0.0,취하,./tables/005/20190801+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2019-08-01
72995,혼합제제,시가현,2019-06-11,25.00,8.0,0.0,취하,./tables/011/20190704+일본산+수입식품+검사현황+일일보고-page-5-table-1.csv,True,2019-07-04
82589,기타수산물가공품,에히메현,2019-01-24,5025.00,1.0,0.0,취하,./tables/038/20190314+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2019-03-14
83040,볶은커피,미에현,2019-01-02,42.00,2.0,0.0,취하,./tables/039/20190311+일본산+수입식품+검사현황+일일보고-page-7-table-1.csv,True,2019-03-11
136125,면류첨가알칼리제,오사카부,2018-09-19,100.00,2.0,0.0,취하,./tables/059/20181128+일일보고-page-6-table-1.csv,True,2018-11-28
138211,아연,오사카부,2018-08-08,31.00,16.0,0.0,취하,./tables/064/20181109+일일보고-page-3-table-1.csv,True,2018-11-09
94050,빌베리추출물(원료성),지바현,2018-03-01,40.00,2.0,0.0,취하,./tables/075/20180917+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2018-09-17
97461,아연,오사카부,2021-06-10,31.00,28.0,0.0,취하,./tables/086/20180730+일본산+수입식품+검사현황+일일보고-page-6-table-1.csv,False,2018-07-30
112302,침출차,가고시마현,2017-12-19,20.00,1.0,0.0,취하,./tables/129/20180123+일본산+수입식품+검사현황+일일보고-page-3-table-1.csv,True,2018-01-23
114270,침출차,가고시마현,2017-11-22,100.00,1.0,0.0,취하,./tables/135/20171227+일본산+수입식품+검사현황+일일보고-page-4-table-1.csv,True,2017-12-27


In [63]:
print(gen_japan.loc[gen_japan['inspection_date'].dt.year==2014].shape)
print(gen_japan.loc[gen_japan['inspection_date'].dt.year==2015].shape)
print(gen_japan.loc[gen_japan['inspection_date'].dt.year==2016].shape)
print(gen_japan.loc[gen_japan['inspection_date'].dt.year==2017].shape)
print(gen_japan.loc[gen_japan['inspection_date'].dt.year==2018].shape)
print(gen_japan.loc[gen_japan['inspection_date'].dt.year==2019].shape)

(16727, 10)
(22648, 10)
(23693, 10)
(26331, 10)
(29086, 10)
(18915, 10)


In [64]:
gen_japan.loc[gen_japan['inspection_date'].dt.year==2015, '세슘'].value_counts()

0.0    22648
Name: 세슘, dtype: int64

In [65]:
tmp = gen_not_japan
tmp.groupby([tmp['제조·생산지'], tmp['inspection_date'].dt.year]).agg({'세슘':'sum'})

,,세슘
제조·생산지,inspection_date,
2015-03-11,2016,0.0
2015-11-24,2016,0.0
2015-12-02,2016,0.0
2015-12-31,2016,0.0
2016-01-02,2016,0.0
2016-01-16,2016,0.0
2016-01-19,2016,0.0
2016-01-20,2016,0.0
2016-01-26,2016,0.0


In [66]:
gen_not_japan.loc[gen_not_japan['제조·생산지'].str[0]=='2']

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
40292,소스류,2016-04-05,2016-03-01,2030.40,0.0,0.0,적합,./tables/256/20160421-page-5-table-1.csv,True,2016-04-21
40293,혼합제제,2016-03-29,2016-03-01,1000.00,0.0,0.0,적합,./tables/256/20160421-page-5-table-1.csv,True,2016-04-21
40294,청주,2016-03-10,2016-03-31,648.00,0.0,0.0,적합,./tables/256/20160421-page-5-table-1.csv,True,2016-04-21
40295,청주,2016-03-01,2016-03-24,1296.00,0.0,0.0,적합,./tables/256/20160421-page-5-table-1.csv,True,2016-04-21
40296,과자(스낵과자),2016-03-22,2016-03-10,288.00,0.0,0.0,적합,./tables/256/20160421-page-4-table-1.csv,True,2016-04-21
40297,혼합제제,2016-04-01,2016-03-10,17.00,0.0,0.0,적합,./tables/256/20160421-page-4-table-1.csv,True,2016-04-21
40298,혼합제제,2016-03-30,2016-03-10,200.00,0.0,0.0,적합,./tables/256/20160421-page-4-table-1.csv,True,2016-04-21
40299,과자(쿠키),2016-03-22,2016-03-11,396.00,0.0,0.0,적합,./tables/256/20160421-page-4-table-1.csv,True,2016-04-21
40300,과자(쿠키),2016-03-22,2016-04-05,600.00,0.0,0.0,적합,./tables/256/20160421-page-4-table-1.csv,True,2016-04-21
40301,즉석조리식품,2016-03-17,2016-01-12,3678.49,0.0,0.0,적합,./tables/256/20160421-page-4-table-1.csv,True,2016-04-21


In [67]:
gen_not_japan.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
141,커피/건조,브라질,2012-09-06,6000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True,2015-01-15
151,커피/건조,브라질,2014-06-26,12000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True,2015-01-15
1298,육두구(육두관)/건조,인도네시아,2014-08-01,3000.0,0.0,0.0,적합,./tables/355/20141226_방사능_검사_현황-page-1-table-2.csv,True,2014-12-26
1340,"회향(소회향,판넬,FoeniculumvulgareMill)/건조",인도,2014-10-01,2000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-3-table-1.csv,True,2014-12-24
1481,커피/건조,브라질,2014-06-26,6000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-2-table-1.csv,True,2014-12-24


In [68]:
idx_supply = ~gen_not_japan['isProductionDate']
gen_not_japan.loc[idx_supply].shape
gen_not_japan.loc[~idx_supply].shape

(1797, 10)

In [69]:
tmp = gen_not_japan.loc[~idx_supply]
tmp.groupby(tmp['제조일자(유통기한)'].dt.year).agg({'세슘':'sum'})

,세슘
제조일자(유통기한),
1997,0.0
1998,0.0
1999,0.0
2000,0.0
2001,0.0
2002,0.0
2003,0.0
2007,0.0
2008,0.0


In [70]:
tmp = fsh_japan
tmp.groupby(tmp['제조일자(유통기한)'].dt.year).agg({'세슘':'sum'})

,세슘
제조일자(유통기한),
2014,0.0
2015,0.0
2016,0.0
2017,0.0
2018,0.0
2019,0.0


In [71]:
tmp = fish_not_japan
tmp.groupby(tmp['제조일자(유통기한)'].dt.year).agg({'세슘':'sum'})

,세슘
제조일자(유통기한),
2014,16.0
2015,1.0
2016,4.0
2017,0.0
2018,4.0
2019,0.0


## 세슘 X 판정

In [72]:
data = fish_meat.loc[fish_meat['세슘']!='불검출']
pd.crosstab(data['세슘'], data['판정'], margins=True).style.background_gradient(cmap='summer_r')

/home/jake/.virtualenvs/workspace/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


판정,적합,적힙,All
세슘,,,
0.0,43692,5,43697
1.0,3,0,3
2.0,4,0,4
14.0,1,0,1
All,43700,5,43705


In [125]:
data = general_japan.loc[general_japan['세슘']!=0]
pd.crosstab(data['세슘'], data['판정'], margins=True).style.background_gradient(cmap='summer_r')

판정,적합,취하,취하*,All
세슘,,,,
1.0,0,4,1,5
2.0,1,3,0,4
3.0,0,1,0,1
4.0,0,0,1,1
8.0,0,1,0,1
16.0,0,1,0,1
26.0,1,0,0,1
28.0,0,1,0,1
All,2,11,2,15


In [124]:
data = gen_not_japan.loc[(gen_not_japan['제조·생산지']=='러시아')&(gen_not_japan['세슘']!=0)]
pd.crosstab(data['세슘'], data['판정'], margins=True).style.background_gradient(cmap='summer_r')

판정,부적합,적합,All
세슘,,,
1.0,0,2,2
2.0,0,2,2
3.0,0,1,1
6.0,0,1,1
7.0,0,1,1
8.0,0,2,2
9.0,0,1,1
10.0,0,1,1
11.0,0,3,3


In [122]:
data = gen_not_japan.loc[gen_not_japan['세슘']!=0]
pd.crosstab(data['세슘'], data['판정'], margins=True).style.background_gradient(cmap='summer_r')

판정,부적합,적합,All
세슘,,,
1.0,0,53,53
2.0,0,85,85
3.0,0,64,64
4.0,0,42,42
5.0,0,15,15
6.0,0,7,7
7.0,0,9,9
8.0,0,9,9
8.8,0,1,1


In [75]:
gen_not_japan.loc[gen_not_japan['세슘']>100].sort_values('세슘', ascending=False)

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
265,과채가공품,독일,2020-05-12,25.00,798.0,0.0,부적합,./tables/096/20180621_기타국가_방사능_검출_현황-page-1-table-1.csv,True,2018-06-21
136,능이/건조,러시아,2020-11-19,100.00,265.0,0.0,부적합,./tables/054/20181220_기타국가_방사능_검출_현황-page-1-table-1.csv,True,2018-12-20
341,잼,이탈리아,2020-04-30,19.00,219.0,0.0,부적합,./tables/133/20180103_기타국가_방사능_검출_현황-page-1-table-1.csv,True,2018-01-03
267,잼,이탈리아,2020-12-31,6.00,120.0,0.0,부적합,./tables/096/20180619_기타국가_방사능_검출_현황-page-1-table-1.csv,True,2018-06-19
228,잼,핀란드,2019-10-22,605.88,114.0,0.0,부적합,./tables/081/20180821_기타국가_방사능_검출_현황-page-1-table-1.csv,True,2018-08-21


일본 수입 식품의 세슘 검출량은 그리 많지 않은 편이다. 그런데 수입되지 않는 경우가 많다.  
객관적인 숫자만 따지면 불합리하다고 볼 수 있다.  
하지만 WTO는 한국의 손을 들어줬다.  

## 일본외 국가 중 검출 순위

In [76]:
gen_japan['세슘'].sum()

98.0

In [121]:
tmp = gen_not_japan.loc[gen_not_japan['세슘']!=0].groupby('제조·생산지').agg({'세슘':'count'}).reset_index()
tmp2 = gen_not_japan.groupby('제조·생산지').agg({'제조일자(유통기한)':'count'}).reset_index()
tmp3 = tmp.merge(tmp2, on='제조·생산지')
tmp3['검출비율'] = tmp3['세슘'] / tmp3['제조일자(유통기한)'] * 100
tmp3.sort_values(['검출비율', '제조일자(유통기한)'], ascending=False)

,제조·생산지,세슘,제조일자(유통기한),검출비율
9,스웨덴,35,35,100.000000
20,키르기스스탄,8,8,100.000000
22,폴란드,8,8,100.000000
15,오스트리아,4,4,100.000000
5,리투아니아,3,3,100.000000
12,슬로바키아,3,3,100.000000
4,루마니아,1,1,100.000000
8,세르비아,1,1,100.000000
13,에스토니아,1,1,100.000000
3,러시아,110,111,99.099099


In [208]:
tmp = gen_not_japan.groupby('제조·생산지').agg({'세슘':'sum'}).reset_index().sort_values('세슘', ascending=False)
tmp

# gen_not_japan.groupby('제조·생산지').agg({'판정':''})
tmp2 = gen_not_japan.loc[gen_not_japan['판정']!='적합'].groupby('제조·생산지').agg({'판정':'count'}).reset_index()
tmp2

tmp3 = tmp.merge(tmp2, on='제조·생산지', how='left').fillna(0)
tmp3.rename(columns={'판정':'부적합판정'}, inplace=True)
tmp3

,제조·생산지,세슘,부적합판정
0,러시아,4700.0,1.0
1,독일,996.0,1.0
2,이탈리아,886.0,2.0
3,중국,778.0,0.0
4,프랑스,506.7,0.0
5,핀란드,486.0,1.0
6,스웨덴,331.0,0.0
7,미국,254.0,0.0
8,벨기에,172.0,0.0
9,키르기스스탄,116.0,0.0


### Export: 국가별 방사능 검출량

In [209]:
tmp3.head(10).to_csv(dir_tableau + 'radio_per_country.tsv', sep='\t', index=False)

In [108]:
tmp2 = gen_not_japan.groupby('제조·생산지').agg({'수입중량(kg)':'sum'}).reset_index().sort_values('수입중량(kg)', ascending=False)

tmp3 = tmp.merge(tmp2, on='제조·생산지')
tmp3['검출비율'] = tmp3['세슘'] / tmp3['수입중량(kg)']
tmp3.sort_values('세슘', ascending=False)

,제조·생산지,세슘,수입중량(kg),검출비율
0,러시아,4700.0,146560.22,0.032069
1,독일,996.0,42691.30,0.023330
2,이탈리아,886.0,6027.15,0.147001
3,중국,778.0,1451836.50,0.000536
4,프랑스,506.7,50126.69,0.010108
5,핀란드,486.0,12410.42,0.039161
6,스웨덴,331.0,70051.20,0.004725
7,미국,254.0,260542.68,0.000975
8,벨기에,172.0,3908.90,0.044002
9,키르기스스탄,116.0,16511.00,0.007026


In [78]:
fsh_not_japan.groupby('제조·생산지').agg({'세슘':'sum'}).reset_index().sort_values('세슘', ascending=False)

,제조·생산지,세슘
83,캐나다,14.0
13,대만,7.0
9,네덜란드,2.0
3,그리스,1.0
64,에스토니아,1.0
70,우루과이,0.0
80,중국홍콩(),0.0
79,중국,0.0
78,일본,0.0
77,인도양,0.0


In [80]:
gen_japan['수입중량(kg)'].sum()

568097471.416

In [81]:
4700/98

47.95918367346939

In [82]:
total = gen_japan.loc[gen_japan['세슘']!=0].shape[0]
canceled = gen_japan.loc[gen_japan['판정']!='적합'].shape[0]
canceled/total*100

86.66666666666667

In [83]:
total = gen_not_japan.loc[gen_not_japan['세슘']!=0].shape[0]
canceled = gen_not_japan.loc[gen_not_japan['판정']!='적합'].shape[0]
canceled/total*100

1.0141987829614605

In [84]:
86.6 / 1

86.6

In [85]:
gen_not_japan.loc[gen_not_japan['제조·생산지']=='러시아', '판정'].value_counts()

적합     110
부적합    1  
Name: 판정, dtype: int64

러시아 > 독일 > 이탈리아 > 중국

## 가장 많이 검출된 식품

In [109]:
gen_not_japan.loc[(gen_not_japan['제조·생산지']=='러시아')&(gen_not_japan['세슘']!=0)].groupby('품목명').agg({'세슘':'sum'})

,세슘
품목명,
건조차가버섯,43.0
건차가버섯,49.0
고형차,3749.0
능이/건조,265.0
능이버섯,90.0
능이버섯(건조),81.0
버섯류,12.0
송이버섯,8.0
액상차,74.0


In [86]:
gen_not_japan.groupby('품목명').agg({'세슘':'sum'}).reset_index().sort_values('세슘', ascending=False)

,품목명,세슘
19,고형차,3993.0
135,잼,1493.0
33,과채가공품,1322.0
58,능이/건조,588.0
24,과․채가공품,238.7
14,건조능이버섯,235.0
59,능이/냉동,171.0
146,차가버섯,156.0
147,차가버섯/건조,140.0
12,건조그물버섯,126.0


In [87]:
gen_not_japan.head()

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
141,커피/건조,브라질,2012-09-06,6000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True,2015-01-15
151,커피/건조,브라질,2014-06-26,12000.0,0.0,0.0,적합,./tables/352/20150115_방사능_검사_현황-page-2-table-1.csv,True,2015-01-15
1298,육두구(육두관)/건조,인도네시아,2014-08-01,3000.0,0.0,0.0,적합,./tables/355/20141226_방사능_검사_현황-page-1-table-2.csv,True,2014-12-26
1340,"회향(소회향,판넬,FoeniculumvulgareMill)/건조",인도,2014-10-01,2000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-3-table-1.csv,True,2014-12-24
1481,커피/건조,브라질,2014-06-26,6000.0,0.0,0.0,적합,./tables/356/20141224_방사능_검사_현황-page-2-table-1.csv,True,2014-12-24


In [88]:
gen_not_japan.loc[gen_not_japan['품목명'].str.contains('차가버섯')]

,품목명,제조·생산지,제조일자(유통기한),수입중량(kg),세슘,요오드,판정,파일,isProductionDate,inspection_date
4,차가버섯/건조,러시아,2019-06-13,3520.0,18.0,0.0,적합,./tables/003/20190809+기타국가+방사능+검출+현황-page-1-table-1.csv,False,2019-08-09
66,차가버섯/건조,러시아,2019-04-01,198.0,40.0,0.0,적합,./tables/024/20190510_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2019-05-10
189,차가버섯/건조,러시아,2018-01-01,220.0,82.0,0.0,적합,./tables/069/20181018_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2018-10-18
207,차가버섯,러시아,2020-08-28,198.0,1.0,0.0,적합,./tables/075/20180914_기타국가_방사능_검출_현황-page-1-table-1.csv,True,2018-09-14
227,차가버섯,러시아,2018-06-28,5000.0,17.0,0.0,적합,./tables/081/20180821_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2018-08-21
240,차가버섯,러시아,2018-04-20,6462.0,11.0,0.0,적합,./tables/088/20180723_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2018-07-23
246,차가버섯,러시아,2018-05-20,15847.0,25.0,0.0,적합,./tables/089/20180716_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2018-07-16
273,차가버섯,러시아,2018-03-20,13308.0,17.0,0.0,적합,./tables/098/20180612_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2018-06-12
321,차가버섯,러시아,2017-11-26,5730.0,29.0,0.0,적합,./tables/124/20180212_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2018-02-12
378,차가버섯,러시아,2017-10-01,12000.0,15.0,0.0,적합,./tables/140/20171206_기타국가_방사능_검출_현황-page-1-table-1.csv,False,2017-12-06


## 2017년 국가별 식품 수입

In [89]:
tmp1 = ['미국', '호주', '중국', '러시아연방', '브라질', '베트남', '캐나다', '필리핀', '태국', '독일', '말레이시아', '일본', '남아프리카 공화국', '인도네시아',
        '세르비아', '스페인', '파라과이', '칠레', '인도', '뉴질랜드',]
tmp2 = ['4,135,085,243', '3,502,017,293', '2,940,087,320', '1,244,154,153', '898,118,191', '502,565,311', '499,625,139', '465,043,977',
        '373,955,910', '304,601,959', '282,674,962', '183,153,879', '178,161,920', '178,016,271', '167,321,654', '146,609,188', '143,258,341',
        '141,256,300', '138,886,909', '134,814,713',]

# food_import = pd.DataFrame(columns=['국가','수입중량(kg)'], data=[tmp1, tmp2])
food_import = pd.DataFrame({'국가':tmp1, '수입중량(kg)':tmp2})
food_import['수입중량(kg)'] = food_import['수입중량(kg)'].str.replace(',','')
food_import['수입중량(kg)'] = food_import['수입중량(kg)'].astype(float)
food_import.sort_values('수입중량(kg)', ascending=False)

,국가,수입중량(kg)
0,미국,4.135085e+09
1,호주,3.502017e+09
2,중국,2.940087e+09
3,러시아연방,1.244154e+09
4,브라질,8.981182e+08
5,베트남,5.025653e+08
6,캐나다,4.996251e+08
7,필리핀,4.650440e+08
8,태국,3.739559e+08
9,독일,3.046020e+08
